<h1 id="header">Segmenting and Clustering Neighborhoods</h1>

Instructions:

To create the dataframe -

• The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

• Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

• More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

• If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

• Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

• In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

<h3 id="header">1. Get the Table of Data as Pandas Dataframe</h3>

In [4]:
# use Beautiful Soup for website scraping

!pip install BeautifulSoup4
!pip install requests

print("Done!")

Done!


In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

# create a list and store data from html in list
data = []
columns = []
table = soup.find(class_='wikitable')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    # this will be the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

# convert to pandas dataframe
boroughs_df = pd.DataFrame(data = data,columns = columns)
boroughs_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3 id="header">2. Clean Data</h3>
• Ignore cells with a borough that is 'Not Assigned'
<br>• Combine neighborhoods with same postal code, separating names with commas
<br>• If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [6]:
# remove 'Not Assigned' boroughs
boroughs_df.drop(boroughs_df[boroughs_df['Borough'] == "Not assigned"].index,axis=0, inplace=True)

boroughs_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [44]:
# combine neighborhoods with same postal code, separating names with commas
boroughs_commas_df = boroughs_df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))

# remove duplicates
boroughs_commas_df = boroughs_commas_df.drop_duplicates()
    
boroughs_commas_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [45]:
# if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
boroughs_commas_df['Neighbourhood'].replace("Not assigned", boroughs_commas_df["Borough"],inplace=True)
boroughs_commas_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3 id="header">3. Print Number of Rows using .shape</h3>

In [46]:
final_df = boroughs_commas_df
final_df.shape

(103, 3)

<h1 id="header">Segmenting and Clustering Neighborhoods Part 2</h1>

Instructions:

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Here is the link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the dataframe with postal code, latitude, and longitude coordinates.

In [10]:
# get data
import pandas as pd
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coordinates = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [11]:
# show info on data
print(coordinates.shape)
coordinates.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
# index data set by Postal Code and merge final_df and coordinates
neighborhood_df = final_df.set_index("Postal Code")
coordinates_df = coordinates.set_index("Postal Code")
neighborhood_coordinates_df = pd.concat([final_df, coordinates_df], axis=1, join='inner')

neighborhood_coordinates_df.index.name = "Postal Code"

print(neighborhood_coordinates_df.shape)
neighborhood_coordinates_df.head()

(0, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,,
